# Análise dos imoveis Balneário Camboriú
## Michel Souza Santana
### Data Science Student
> Data: 16/01/2022

##### Objetivos
* Capturar código fonte da página via url
* Do código fonte, extrair dados que possam ter serventia para gerar informações
* Organizar os dados capturados em uma tabela (DataFrame)
* Realizar limpeza dos dados preparando-os para análise

### Importando as bibliotecas necessárias para execução da análise e preparação 

In [189]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import re

### Requisitando o código fonte da página com os dados que nos interessa

In [190]:
html = urlopen('https://www.tresas.com.br/todos-os-imoveis')
cf = bs(html.read(), 'html.parser')
dados_imoveis = cf.findAll('a', {'class': 'listing-card-2 max-400 w-inline-block'})
dados_imoveis[:2]

[<a class="listing-card-2 max-400 w-inline-block" href="/imoveis/apartamento-aconchegante-e-moderno-no-bairro-pioneiros-em-balneario-camboriu"><div class="listing-2-card-top"><img alt="" class="listing-card-2-photo" height="400" loading="lazy" sizes="(max-width: 479px) 88vw, (max-width: 991px) 380px, (max-width: 1439px) 28vw, 400px" src="https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62b50d4ec9f9c76f72a7b1a3_Untitled%20design%20(57).png" srcset="https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62b50d4ec9f9c76f72a7b1a3_Untitled%20design%20(57)-p-500.png 500w, https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62b50d4ec9f9c76f72a7b1a3_Untitled%20design%20(57)-p-800.png 800w, https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62b50d4ec9f9c76f72a7b1a3_Untitled%20design%20(57)-p-1080.png 1080w, https://uploads-ssl.webflow.com/629007f9f06b5eda2376045d/62b50d4ec9f9c76f72a7b1a3_Untitled%20design%20(57)-p-1600.png 1600w, https://uploads-ssl.webflow.com/629007f9f06b

### Capturando os valores das tags requisitadas para montar nosso dataframe

In [191]:
"""imoveis = []
for dados in dados_imoveis:
    novos_dados = []
    novos_dados.append(dados.find('div', {'class': 'text-large'})) #descrição
    novos_dados.append(dados.find('div', {'class': 'listing-2-price-text'})) #preço
    novos_dados.append(dados.find('div', {'class': 'text-medium margin-top-4'})) #localização
    novos_dados.append(dados.find('div', {'class': 'listing-2-card-features'})) #caracteristicas do imóvel
    novos_dados.append(dados.img['src']) #link imagem
    imoveis.append(novos_dados)"""
    
#CODIGO MELHORADO I
imoveis = []
for dados in dados_imoveis:
    novos_dados = []
    novos_dados.append(dados.find('div', {'class': 'text-large'}).get_text()) #descrição
    novos_dados.append(dados.find('div', {'class': 'listing-2-price-text'}).get_text()) #preço
    novos_dados.append(dados.find('div', {'class': 'text-medium margin-top-4'}).get_text()) #localização
    novos_dados.append(dados.find('div', {'class': 'listing-2-card-features'}).get_text()) #caracteristicas do imóvel
    novos_dados.append(dados.img['src']) #link imagem
    imoveis.append(novos_dados)

### Iniciando a montagem do dataframe

In [192]:
df = pd.DataFrame(imoveis, columns=['descricao_do_imovel', 'preco', 'localizacao', 'caracteristicas_do_imovel', 'link_da_imagem']) 
df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,"R$ 2.750.000,00","Pioneiros, Balneário Camboriú - SC",3 Suítes4 Banheiros131m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,"R$ 9.000.000,00","Centro, Balneário Camboriú - SC",4 Suítes4 Banheiros490m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,"R$ 1.150.000,00","Rua 2350, 118. Centro -Balneário Camboriú",2 Suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...","R$ 2.989.188,00","Rua 1201, 432. Centro - Balneário Camboriú","4 Suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC","R$ 4.879.476,00","Rua 1950, 216 - Centro - Balneário Camboriú",4 Suítes240 m² de Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


## Iniciando o processo de limpeza e preparação dos dados

> Nesse passo, mesmo que não integraumente necessário, poderemos vir a utilizar algumas funções e ou técnicas para conhecimento e prática.

* Verificando informações quantitativas da tabela (Colunas X Linhas)

In [193]:
df.shape

(60, 5)

* Tratando os dados ausentes (Caso existam)

In [194]:
df.isnull().sum()

descricao_do_imovel          0
preco                        0
localizacao                  0
caracteristicas_do_imovel    0
link_da_imagem               0
dtype: int64

>> Realizada verificação de dados nulos e em sua soma, percebe-se que estes não existem nesta tabela

>> Caso existicem dados nulos rodariamos o seguinte comando para trata-los:

OBS: Sempre levar em consideração o quanto a retirada desses conjuntos de dados poderá afetar o resultado esperado.

In [195]:
df = df.dropna()
df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,"R$ 2.750.000,00","Pioneiros, Balneário Camboriú - SC",3 Suítes4 Banheiros131m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,"R$ 9.000.000,00","Centro, Balneário Camboriú - SC",4 Suítes4 Banheiros490m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,"R$ 1.150.000,00","Rua 2350, 118. Centro -Balneário Camboriú",2 Suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...","R$ 2.989.188,00","Rua 1201, 432. Centro - Balneário Camboriú","4 Suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC","R$ 4.879.476,00","Rua 1950, 216 - Centro - Balneário Camboriú",4 Suítes240 m² de Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


* Caso julgue serem os dados de grande importância, podemos simplesmente preencher as lacunas substituindo os nulos por outro valor.

In [196]:
df = df.fillna(0)
df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,"R$ 2.750.000,00","Pioneiros, Balneário Camboriú - SC",3 Suítes4 Banheiros131m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,"R$ 9.000.000,00","Centro, Balneário Camboriú - SC",4 Suítes4 Banheiros490m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,"R$ 1.150.000,00","Rua 2350, 118. Centro -Balneário Camboriú",2 Suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...","R$ 2.989.188,00","Rua 1201, 432. Centro - Balneário Camboriú","4 Suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC","R$ 4.879.476,00","Rua 1950, 216 - Centro - Balneário Camboriú",4 Suítes240 m² de Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


* Removendo duplicados

In [197]:
df.duplicated().sum()

0

>> Em nossa tabela não há dados duplicados, más caso haja e seja se seu interesse removê-lo por algum motivo, seria feito assim:

In [198]:
df.drop_duplicates().head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,"R$ 2.750.000,00","Pioneiros, Balneário Camboriú - SC",3 Suítes4 Banheiros131m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,"R$ 9.000.000,00","Centro, Balneário Camboriú - SC",4 Suítes4 Banheiros490m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,"R$ 1.150.000,00","Rua 2350, 118. Centro -Balneário Camboriú",2 Suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...","R$ 2.989.188,00","Rua 1201, 432. Centro - Balneário Camboriú","4 Suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC","R$ 4.879.476,00","Rua 1950, 216 - Centro - Balneário Camboriú",4 Suítes240 m² de Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


>> Havendo necessidade de uma maior precizão de quais dados duplicados deseja eliminar, fica melhor assim:

In [199]:
df.drop_duplicates(['descricao_do_imovel']).head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,"R$ 2.750.000,00","Pioneiros, Balneário Camboriú - SC",3 Suítes4 Banheiros131m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,"R$ 9.000.000,00","Centro, Balneário Camboriú - SC",4 Suítes4 Banheiros490m² Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,"R$ 1.150.000,00","Rua 2350, 118. Centro -Balneário Camboriú",2 Suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...","R$ 2.989.188,00","Rua 1201, 432. Centro - Balneário Camboriú","4 Suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC","R$ 4.879.476,00","Rua 1950, 216 - Centro - Balneário Camboriú",4 Suítes240 m² de Área Privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


* Aqui, estarei colocando todos os dados da coluna 'caracteristicas_do imóvel' em minusculas para facilitar tratamento futuro.

In [200]:
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.lower()
df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,"R$ 2.750.000,00","Pioneiros, Balneário Camboriú - SC",3 suítes4 banheiros131m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,"R$ 9.000.000,00","Centro, Balneário Camboriú - SC",4 suítes4 banheiros490m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,"R$ 1.150.000,00","Rua 2350, 118. Centro -Balneário Camboriú",2 suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...","R$ 2.989.188,00","Rua 1201, 432. Centro - Balneário Camboriú","4 suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC","R$ 4.879.476,00","Rua 1950, 216 - Centro - Balneário Camboriú",4 suítes240 m² de área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


* Substituindo valores

In [201]:
df['preco'] = df['preco'].str.replace('[R$ .]', '')
df['preco'] = df['preco'].str.replace(',', '.')
df.head()

/tmp/ipykernel_44995/3323248047.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preco'] = df['preco'].str.replace('[R$ .]', '')


,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,2750000.00,"Pioneiros, Balneário Camboriú - SC",3 suítes4 banheiros131m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,9000000.00,"Centro, Balneário Camboriú - SC",4 suítes4 banheiros490m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,1150000.00,"Rua 2350, 118. Centro -Balneário Camboriú",2 suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...",2989188.00,"Rua 1201, 432. Centro - Balneário Camboriú","4 suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC",4879476.00,"Rua 1950, 216 - Centro - Balneário Camboriú",4 suítes240 m² de área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


In [202]:
df['preco'] = df['preco'].replace(',', '')
new = df['preco'].str.split('.', expand=True)
df['preco'] = new[0]
df.head()

,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,2750000,"Pioneiros, Balneário Camboriú - SC",3 suítes4 banheiros131m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,9000000,"Centro, Balneário Camboriú - SC",4 suítes4 banheiros490m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,1150000,"Rua 2350, 118. Centro -Balneário Camboriú",2 suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...",2989188,"Rua 1201, 432. Centro - Balneário Camboriú","4 suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC",4879476,"Rua 1950, 216 - Centro - Balneário Camboriú",4 suítes240 m² de área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


* Alterando caracteres na string de localizacao para facilitar tratamento futuro

In [203]:
df['localizacao'] = df['localizacao'].str.replace('[-]', ',')
df.head()

/tmp/ipykernel_44995/3583796940.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['localizacao'] = df['localizacao'].str.replace('[-]', ',')


,descricao_do_imovel,preco,localizacao,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,2750000,"Pioneiros, Balneário Camboriú , SC",3 suítes4 banheiros131m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,9000000,"Centro, Balneário Camboriú , SC",4 suítes4 banheiros490m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,1150000,"Rua 2350, 118. Centro ,Balneário Camboriú",2 suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...",2989188,"Rua 1201, 432. Centro , Balneário Camboriú","4 suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC",4879476,"Rua 1950, 216 , Centro , Balneário Camboriú",4 suítes240 m² de área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


* Separando dados de localizacao

In [204]:
new = df['localizacao'].str.split(',', expand=True)
df['bairro'] = new[0]
df['cidade'] = new[1]
df['estado'] = new[2]
df.drop(columns=['localizacao'], inplace=True)
df.head()


,descricao_do_imovel,preco,caracteristicas_do_imovel,link_da_imagem,bairro,cidade,estado
0,Apartamento Aconchegante e Moderno no Bairro P...,2750000,3 suítes4 banheiros131m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...,Pioneiros,Balneário Camboriú,SC
1,Apartamento com 4 suítes no Condomínio Ibiza T...,9000000,4 suítes4 banheiros490m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...,Centro,Balneário Camboriú,SC
2,Apartamento Mobiliado em Balneário Camboriú,1150000,2 suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...,Rua 2350,118. Centro,Balneário Camboriú
3,"Apartamento no Diamond Crystal Tower, Centro d...",2989188,"4 suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...,Rua 1201,432. Centro,Balneário Camboriú
4,"Cobertura Duplex no Edifício Evely, Centro de BC",4879476,4 suítes240 m² de área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...,Rua 1950,216,Centro


* Reorganizando ordem das colunas

In [205]:
df = df[['descricao_do_imovel', 'bairro', 'cidade', 'estado', 'preco', 'caracteristicas_do_imovel', 'link_da_imagem']]
df.head()

,descricao_do_imovel,bairro,cidade,estado,preco,caracteristicas_do_imovel,link_da_imagem
0,Apartamento Aconchegante e Moderno no Bairro P...,Pioneiros,Balneário Camboriú,SC,2750000,3 suítes4 banheiros131m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
1,Apartamento com 4 suítes no Condomínio Ibiza T...,Centro,Balneário Camboriú,SC,9000000,4 suítes4 banheiros490m² área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...
2,Apartamento Mobiliado em Balneário Camboriú,Rua 2350,118. Centro,Balneário Camboriú,1150000,2 suítes,https://uploads-ssl.webflow.com/629007f9f06b5e...
3,"Apartamento no Diamond Crystal Tower, Centro d...",Rua 1201,432. Centro,Balneário Camboriú,2989188,"4 suítes173,79m²",https://uploads-ssl.webflow.com/629007f9f06b5e...
4,"Cobertura Duplex no Edifício Evely, Centro de BC",Rua 1950,216,Centro,4879476,4 suítes240 m² de área privativa,https://uploads-ssl.webflow.com/629007f9f06b5e...


* Dividindo as caracteristicas por partes e preservando apenas o quantitativo

In [206]:
df['preco'] = df['preco'].str.replace('Sobconsulta', '')
df['preco'] = df['preco'].str.replace('SobConsulta', '')
df['preco'] = df['preco'].str.replace('Apartirde', '')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace(' m2', ',')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('suítes', ',')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('suíte', ',') 
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('banheiros', ',')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('m² área privativa', '')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace(' m² área total', '')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('m²', '')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('a partir de ', '')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace(' de terreno', '')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('  de área privativa', '')
df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('200.00', '200')

new = df['caracteristicas_do_imovel'].str.split('[,]', expand=True)
df['suites'] = new[0] 
df['banheiros'] = new[1] 
df['m2'] = new[2] 
df.drop(columns=['caracteristicas_do_imovel'], inplace=True)
df.head()

/tmp/ipykernel_44995/2198212394.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['caracteristicas_do_imovel'] = df['caracteristicas_do_imovel'].str.replace('200.00', '200')


,descricao_do_imovel,bairro,cidade,estado,preco,link_da_imagem,suites,banheiros,m2
0,Apartamento Aconchegante e Moderno no Bairro P...,Pioneiros,Balneário Camboriú,SC,2750000,https://uploads-ssl.webflow.com/629007f9f06b5e...,3,4,131
1,Apartamento com 4 suítes no Condomínio Ibiza T...,Centro,Balneário Camboriú,SC,9000000,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,4,490
2,Apartamento Mobiliado em Balneário Camboriú,Rua 2350,118. Centro,Balneário Camboriú,1150000,https://uploads-ssl.webflow.com/629007f9f06b5e...,2,,None
3,"Apartamento no Diamond Crystal Tower, Centro d...",Rua 1201,432. Centro,Balneário Camboriú,2989188,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,173,79
4,"Cobertura Duplex no Edifício Evely, Centro de BC",Rua 1950,216,Centro,4879476,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,240,None


In [207]:
df = df.fillna(0)
df.head()

,descricao_do_imovel,bairro,cidade,estado,preco,link_da_imagem,suites,banheiros,m2
0,Apartamento Aconchegante e Moderno no Bairro P...,Pioneiros,Balneário Camboriú,SC,2750000,https://uploads-ssl.webflow.com/629007f9f06b5e...,3,4,131
1,Apartamento com 4 suítes no Condomínio Ibiza T...,Centro,Balneário Camboriú,SC,9000000,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,4,490
2,Apartamento Mobiliado em Balneário Camboriú,Rua 2350,118. Centro,Balneário Camboriú,1150000,https://uploads-ssl.webflow.com/629007f9f06b5e...,2,,0
3,"Apartamento no Diamond Crystal Tower, Centro d...",Rua 1201,432. Centro,Balneário Camboriú,2989188,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,173,79
4,"Cobertura Duplex no Edifício Evely, Centro de BC",Rua 1950,216,Centro,4879476,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,240,0


* Tratando valores iregulares nas colunas suites, banheiros e m2

In [208]:
df['preco'] = df['preco'].replace(',', '')
new = df['preco'].str.split('.', expand=True)
df['preco'] = new[0]
df.head()

,descricao_do_imovel,bairro,cidade,estado,preco,link_da_imagem,suites,banheiros,m2
0,Apartamento Aconchegante e Moderno no Bairro P...,Pioneiros,Balneário Camboriú,SC,2750000,https://uploads-ssl.webflow.com/629007f9f06b5e...,3,4,131
1,Apartamento com 4 suítes no Condomínio Ibiza T...,Centro,Balneário Camboriú,SC,9000000,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,4,490
2,Apartamento Mobiliado em Balneário Camboriú,Rua 2350,118. Centro,Balneário Camboriú,1150000,https://uploads-ssl.webflow.com/629007f9f06b5e...,2,,0
3,"Apartamento no Diamond Crystal Tower, Centro d...",Rua 1201,432. Centro,Balneário Camboriú,2989188,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,173,79
4,"Cobertura Duplex no Edifício Evely, Centro de BC",Rua 1950,216,Centro,4879476,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,240,0


In [209]:
for i in range(len(df['preco'])):
    if df['preco'][i] == '' or df['preco'][i] == ' ':
        df['preco'][i] = 0

In [210]:
for i in range(len(df['suites'])):
    if df['suites'][i] == '' or df['suites'][i] == ' ':
        df['suites'][i] = 0

In [211]:
for i in range(len(df['banheiros'])):
    if df['banheiros'][i] == '' or df['banheiros'][i] == ' ':
        df['banheiros'][i] = 0

In [212]:
for i in range(len(df['m2'])):
    if df['m2'][i] == '' or df['m2'][i] == ' ':
        df['m2'][i] = 0

In [213]:
df['suites'] = df['suites'].astype('int64')
df['banheiros'] = df['banheiros'].astype('int64')
df['m2'] = df['m2'].astype('int64')

In [214]:
tot = int(len(df['banheiros']))
for i in range(tot):
    if df['banheiros'][i] >= 10:
        df['m2'][i] = df['banheiros'][i]
        df['banheiros'][i] = 0
        
df.head()

/tmp/ipykernel_44995/338831823.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['m2'][i] = df['banheiros'][i]
/tmp/ipykernel_44995/338831823.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['banheiros'][i] = 0


,descricao_do_imovel,bairro,cidade,estado,preco,link_da_imagem,suites,banheiros,m2
0,Apartamento Aconchegante e Moderno no Bairro P...,Pioneiros,Balneário Camboriú,SC,2750000,https://uploads-ssl.webflow.com/629007f9f06b5e...,3,4,131
1,Apartamento com 4 suítes no Condomínio Ibiza T...,Centro,Balneário Camboriú,SC,9000000,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,4,490
2,Apartamento Mobiliado em Balneário Camboriú,Rua 2350,118. Centro,Balneário Camboriú,1150000,https://uploads-ssl.webflow.com/629007f9f06b5e...,2,0,0
3,"Apartamento no Diamond Crystal Tower, Centro d...",Rua 1201,432. Centro,Balneário Camboriú,2989188,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,0,173
4,"Cobertura Duplex no Edifício Evely, Centro de BC",Rua 1950,216,Centro,4879476,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,0,240


In [215]:
df['preco'] = df['preco'].astype(float)

In [216]:
df.dtypes

descricao_do_imovel     object
bairro                  object
cidade                  object
estado                  object
preco                  float64
link_da_imagem          object
suites                   int64
banheiros                int64
m2                       int64
dtype: object

In [217]:
df.head()

,descricao_do_imovel,bairro,cidade,estado,preco,link_da_imagem,suites,banheiros,m2
0,Apartamento Aconchegante e Moderno no Bairro P...,Pioneiros,Balneário Camboriú,SC,2750000.0,https://uploads-ssl.webflow.com/629007f9f06b5e...,3,4,131
1,Apartamento com 4 suítes no Condomínio Ibiza T...,Centro,Balneário Camboriú,SC,9000000.0,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,4,490
2,Apartamento Mobiliado em Balneário Camboriú,Rua 2350,118. Centro,Balneário Camboriú,1150000.0,https://uploads-ssl.webflow.com/629007f9f06b5e...,2,0,0
3,"Apartamento no Diamond Crystal Tower, Centro d...",Rua 1201,432. Centro,Balneário Camboriú,2989188.0,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,0,173
4,"Cobertura Duplex no Edifício Evely, Centro de BC",Rua 1950,216,Centro,4879476.0,https://uploads-ssl.webflow.com/629007f9f06b5e...,4,0,240


In [218]:
for i in range(len(df['suites'])):
    if df['preco'][i] <=0:
        df['preco'][i] = df['preco'].mean()
    """if df['suites'][i] <=0:
        df['suites'][i] = df['suites'].mean()"""
    if df['banheiros'][i] <=0:
        df['banheiros'][i] = df['banheiros'].mean()
    if df['m2'][i] <=0:
        df['m2'][i] = df['m2'].mean()

/tmp/ipykernel_44995/3786214021.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['banheiros'][i] = df['banheiros'].mean()
/tmp/ipykernel_44995/3786214021.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['m2'][i] = df['m2'].mean()
/tmp/ipykernel_44995/3786214021.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preco'][i] = df['preco'].mean()


In [219]:
df = df[['descricao_do_imovel', 'bairro', 'cidade', 'estado', 'link_da_imagem', 'preco', 'suites', 'banheiros', 'm2']]
df.head()

,descricao_do_imovel,bairro,cidade,estado,link_da_imagem,preco,suites,banheiros,m2
0,Apartamento Aconchegante e Moderno no Bairro P...,Pioneiros,Balneário Camboriú,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,2750000.0,3,4,131
1,Apartamento com 4 suítes no Condomínio Ibiza T...,Centro,Balneário Camboriú,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,9000000.0,4,4,490
2,Apartamento Mobiliado em Balneário Camboriú,Rua 2350,118. Centro,Balneário Camboriú,https://uploads-ssl.webflow.com/629007f9f06b5e...,1150000.0,2,3,123
3,"Apartamento no Diamond Crystal Tower, Centro d...",Rua 1201,432. Centro,Balneário Camboriú,https://uploads-ssl.webflow.com/629007f9f06b5e...,2989188.0,4,3,173
4,"Cobertura Duplex no Edifício Evely, Centro de BC",Rua 1950,216,Centro,https://uploads-ssl.webflow.com/629007f9f06b5e...,4879476.0,4,3,240


In [220]:
df_corr = df.corr()
df_corr.style.background_gradient(cmap='coolwarm')

,preco,suites,banheiros,m2
preco,1.000000,-0.065756,0.552760,0.262609
suites,-0.065756,1.000000,-0.133500,-0.138547
banheiros,0.552760,-0.133500,1.000000,0.320695
m2,0.262609,-0.138547,0.320695,1.000000


* Calculando à média de preço dos imóveis

In [221]:
med = df['preco'].mean()
maxI = df['preco'].max()
minI = df['preco'].min()
print(f'A média de preço dos imóveis é: R${med}')
print(f'O imóvel com maior custo é de: R${maxI}')
print(f'O imóvel com menor custo é de: R${minI}')


A média de preço dos imóveis é: R$5063389.323505324
O imóvel com maior custo é de: R$25400000.0
O imóvel com menor custo é de: R$790000.0


In [222]:
df_order_m2 = df.sort_values(by=['m2'], ascending=False)
df_order_m2.head()

,descricao_do_imovel,bairro,cidade,estado,link_da_imagem,preco,suites,banheiros,m2
1,Apartamento com 4 suítes no Condomínio Ibiza T...,Centro,Balneário Camboriú,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,9000000.00,4,4,490
47,Apartamento no Centro de Balneário Camboriú f...,Centro de Balneário Camboriú,0,0,https://uploads-ssl.webflow.com/629007f9f06b5e...,5850000.00,4,6,384
35,"Cobertura Duplex, no condomínio Brava Center R...",Praia Brava,Itajaí,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,10300000.00,4,4,369
28,"Apartamento no Condomínio Grand Place Tower, B...",Centro,Balneário Camboriú,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,4943000.00,4,5,348
18,"Apartamento no Sapphire Tower, com 4 a 5 quart...",Avenida Brasil,0,0,https://uploads-ssl.webflow.com/629007f9f06b5e...,4818430.15,3,3,285


In [223]:
df.describe()

,preco,suites,banheiros,m2
count,6.000000e+01,60.000000,60.000000,60.000000
mean,5.063389e+06,10.983333,3.850000,176.916667
std,4.260102e+06,43.707228,1.005493,72.812341
min,7.900000e+05,0.000000,2.000000,64.000000
25%,2.723113e+06,3.000000,3.000000,136.500000
50%,4.013037e+06,4.000000,4.000000,158.500000
75%,5.887500e+06,4.000000,4.000000,190.000000
max,2.540000e+07,300.000000,7.000000,490.000000


In [224]:
mais_caro = df.loc[df['preco'] == maxI]
mais_caro

,descricao_do_imovel,bairro,cidade,estado,link_da_imagem,preco,suites,banheiros,m2
53,Cobertura na Barra Sul em Balneário Camboriú c...,Barra Sul,Balneário Camboriú,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,25400000.0,6,6,162


In [225]:
df.to_csv('imoveisBC.csv')

In [226]:
df

,descricao_do_imovel,bairro,cidade,estado,link_da_imagem,preco,suites,banheiros,m2
0,Apartamento Aconchegante e Moderno no Bairro P...,Pioneiros,Balneário Camboriú,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,2.750000e+06,3,4,131
1,Apartamento com 4 suítes no Condomínio Ibiza T...,Centro,Balneário Camboriú,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,9.000000e+06,4,4,490
2,Apartamento Mobiliado em Balneário Camboriú,Rua 2350,118. Centro,Balneário Camboriú,https://uploads-ssl.webflow.com/629007f9f06b5e...,1.150000e+06,2,3,123
3,"Apartamento no Diamond Crystal Tower, Centro d...",Rua 1201,432. Centro,Balneário Camboriú,https://uploads-ssl.webflow.com/629007f9f06b5e...,2.989188e+06,4,3,173
4,"Cobertura Duplex no Edifício Evely, Centro de BC",Rua 1950,216,Centro,https://uploads-ssl.webflow.com/629007f9f06b5e...,4.879476e+06,4,3,240
5,"Apartamento com Vista em B.Camboriú, no Timele...",Avenida Alvin Bauer,159,Centro,https://uploads-ssl.webflow.com/629007f9f06b5e...,3.857000e+06,4,4,181
6,Apartamento com vista panorâmica e 3 suítes na...,Praia Brava,Itajaí,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,6.386291e+06,3,3,168
7,Apartamento no condomínio Cidade Jardim em Bal...,Centro,Balneário Camboriú,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,3.000000e+06,3,4,125
8,"Apartamento com 3 Suítes, 227 m² - Marina de B...",Centro,Balneário Camboriú,SC,https://uploads-ssl.webflow.com/629007f9f06b5e...,5.250000e+06,3,4,227
9,Apartamento quadra mar no condomínio Esquina B...,Balneário Camboriú,SC,0,https://uploads-ssl.webflow.com/629007f9f06b5e...,3.600000e+06,3,4,127
